In [1]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures
import ray as rayt

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-10-02 08:51:23,314 - utils - NumExpr defaulting to 4 threads.


## Ray tracing

Requisites:
* Raw files (blensor ouputs) shoul be placed in the root of the directory
* Raw files should be named `s1*.npy`, `s2*.npy`, etc
* Create a file with sensor positions in the root with name `scanner_pos.txt`. It should contain 4 columns:
    * sensor name i.e. `s1`, `s2`, etc
    * x coordinate
    * y coordinate
    * z coordinate

In [23]:
mockname = 'single_combined_lite'
voxel_size = 0.05
Nipr, Nopr, Nlpc, Npt, volume = rayt.PCdensity(mockname, voxel_size)

PR volume: 	 494.9100000000002


100%|██████████| 1125/1125 [00:00<00:00, 3810.10it/s]

# colide with PR: 	 6381
# Dont colide with PR: 	 1494
# colide with leaves: 	 3446
# pass trhough PR: 	 2935


In [41]:
print(Npt / volume)
print(Nlpc / volume)
print(Nipr / volume)

5.930371178598127
6.962882140187102
12.893253318785229


In [58]:


points = tree.T[5:8].T
dens = []

bins = np.linspace(0.08, 0.5, 20)
for i in bins:
    dens.append([i, lad.get_weigths(points, voxel_size=i)])

dens = np.array(dens)

In [59]:
mockname = 'single_combined_lite'
mockdir = os.path.join(_data, mockname)
segtrees_dir = os.path.join(mockdir, 'toy_trees')
m3count_files = glob.glob(os.path.join(segtrees_dir, 'm3count*.npy'))

dens_counts = []
for m3 in m3count_files:
    vs = float('.'.join(m3.split('/')[-1].split('_')[-1].split('.')[:-1]))
    volume = vs**3
    m3count = np.load(m3)
    dens_ = m3count/volume
    # print(vs, np.sum(dens != 0))
    mean = np.median(dens_[dens_ != 0])
    dens_counts.append([vs, mean])

dens_counts = np.array(dens_counts)
dens_counts = dens_counts[np.argsort(dens_counts[:, 0])]

In [60]:
plt.plot(dens[:,0], dens[:,1], label='LPC')
plt.plot(dens_counts[:,0], dens_counts[:,1], label='BPT')
plt.legend()

In [44]:
#
mockname = 'single_combined_lite'
debug = False
# downsample_debug = None
downsample = None
voxel_size = 0.20

m3att, meshfile, bia, voxk, lia, ws, resdir, tree = lad.get_lad20(mockname, debug=debug, 
                                downsample=downsample, voxel_size=voxel_size)

/home/omar/myproj/planttech/data/single_combined_lite/mesh.ply
size LPC: 	 3446
size new LPC: 	 3446
Stage 1: 0.0036020229999991216
Mean distance between points: 	 0.034093854484334676
STD distance between points: 	 0.037555334834547156
Stage 2: 0.08828863899999817
max --> [47, 45, 29]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (48, 46, 30)
ray tracker voxel dimensions: 	 (48, 46, 30)
Stage 3: 0.4913363240000024
Number of voxels ocupied by points cloud: 	 2355
Number of voxels ocupied by beam points cloud: 	 56554
Total number of voxels in plant regions: 	 66240
Number of voxels with attribute 1: 	 2355
Number of voxels with attribute 2: 	 54199
Number of voxels with attribute 3: 	 9686
Stage 4: 0.017128887000005477
Stage 5: 0.17037932400000244


In [92]:
la = lad.mesh_leaf_area(meshfile)
lpcres = 0.0090 * 1.1
# std = 0.0038
# minvox = (lpcres-std)/la
meanvox = (lpcres)/la
# maxvox = (lpcres+std)/la
# print(minvox, meanvox, maxvox)
print(meanvox)

0.23759809921520628


In [11]:
show=False
klia=False
use_true_lia=True

alphas_k = lad.alpha_k(bia, voxk, lia, ws, resdir, meshfile, 
                        show=show, klia=klia, use_true_lia=use_true_lia)

In [12]:

kbins = 1
# lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
# lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)
lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
# lads_w = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0, mean_weights_k=alphas_k[:,7])
kmax = m3att.shape[2]
lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

In [13]:
fig = plt.figure(figsize=(4, 6))
# plt.plot(lads_min[0:,1], lads_min[0:,0], marker='*', label='Estimated min')
# plt.plot(lads_max[0:,1], lads_max[0:,0], marker='*', label='Estimated max')
plt.plot(lads_mesh[0:,1], lads_mesh[0:,0], marker='s', c='k', label='Truth')
plt.plot(lads_mid[0:,1], lads_mid[0:,0], marker='*', label='Estimated mid')
plt.plot(lads_0[0:,1], lads_0[0:,0], marker='*', label='Estimated 0')
# plt.plot(lads_w[0:,1], lads_w[0:,0], marker='*', label='Estimated weighted')

plt.xlabel(r'LAD ($m^2/m^3$)')
plt.ylabel(r'Height ($m$)')
# plt.grid()
plt.legend()
# plt.show()
# plt.xlim(-2,2)

mockdir = os.path.join(_data, mockname)
resdir_name = '%s_%s' %('results', mockname)
resdir = os.path.join(mockdir, resdir_name)
savefig = os.path.join(resdir, 'LADs.png')
plt.savefig(savefig, dpi=200, bbox_inches='tight')

# Dev Zone...

In [101]:
# tree.shape
points = tree.T[5:8].T
pcd = lad.points2pcd(points)
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

In [102]:
dist = []
for i in range(len(pcd.points)):
    [k, idx, d2] = pcd_tree.search_knn_vector_3d(pcd.points[i], 2)
    # print(pcd.points[i], pcd.points[idx[0]], d2[0])
    dist.append(d2[1])


In [103]:
fig = plt.figure(figsize=(10,6))
plt.hist(np.array(dist), 40)
mean = np.mean(np.array(dist))
plt.axvline(mean, lw=2, ls='--', c='k', label=mean)
plt.legend()
plt.show()